<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Warto%C5%9Bci_odstaj%C4%85ce/colabs/Namierzanie_outlayersow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1565, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 1565 (delta 179), reused 169 (delta 113), pack-reused 1307
Receiving objects: 100% (1565/1565), 6.00 MiB | 26.38 MiB/s, done.
Resolving deltas: 100% (946/946), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Wartości_odstające

Branch 'Wartości_odstające' set up to track remote branch 'Wartości_odstające' from 'origin'.
Switched to a new branch 'Wartości_odstające'


In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=019895abb00c5b22341c762d6b4eef3850a27692cfe6b656e2a152b69338d376
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-14 14:07:05--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [5]:
from lib.pyspark_init import create_spark_context, add_kaggle_data
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

In [6]:
spark = create_spark_context()
spark

In [34]:
%%time
data = spark.read.parquet(JOINED_DATA)
data.show(100)

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|nm0000034|[tt0032636, tt003...|[movie, movie, mo...|[Invisible Stripe...|[0, 0, 0, 0, 0, 0...|     1939|     \N|[Action, Crime, D...|   actor|[["Tim Taylor"], ...|      William Holden|tt0043014,tt00463...|
|nm0000068|[tt0038613, tt002...|[movie, movie, mo...|[Home, Sweet Homi...|[0, 0, 0, 0, 0, 0...|     1931|     \N|[Adventure, Comed...|   actor|[["Lt. Bill Smith...|      Ra

In [12]:
data.createOrReplaceTempView("data");

In [26]:
spark.sql("""select COUNT( DISTINCT nconst)as IDs, COUNT(nconst) as l_aktorow
FROM data""").show()
#wniosek - nie ma duplikatów

+-------+---------+
|    IDs|l_aktorow|
+-------+---------+
|2013276|  2013276|
+-------+---------+



In [35]:
#To samo dla pozostałych kolumn
spark.sql("""select COUNT( DISTINCT category) as category, COUNT( DISTINCT primaryName) as primaryName, COUNT(primaryName) as l_wystapien
FROM data""").show()

+--------+-----------+-----------+
|category|primaryName|l_wystapien|
+--------+-----------+-----------+
|       2|    1806848|    2013276|
+--------+-----------+-----------+



In [49]:
spark.sql("""SELECT primaryName, COUNT(*) as l_wystapien
FROM data
GROUP BY primaryName
HAVING COUNT(*) > 1
ORDER BY l_wystapien DESC""").show(150)

+---------------+-----------+
|    primaryName|l_wystapien|
+---------------+-----------+
|           Alex|        120|
|           Anna|        113|
|          Angel|         95|
|          Maria|         84|
|        Sabrina|         84|
|         Nicole|         80|
|        Jessica|         78|
|       Michelle|         78|
|            Eva|         77|
|         Monica|         77|
|         Sandra|         73|
|          Jenny|         72|
|           Mike|         72|
|        Vanessa|         71|
|          David|         69|
|          Chris|         68|
|           Lisa|         67|
|        Jasmine|         66|
|        Claudia|         65|
|          Amber|         64|
|          Julia|         63|
|         Andrea|         62|
|         Daniel|         61|
|           Gina|         61|
|          James|         60|
|           Lola|         60|
|         Carmen|         60|
|          Nikki|         59|
|       Samantha|         58|
|        Charlie|         58|
|         

In [47]:
spark.sql("""SELECT * from data where primaryName = 'Alex'""").show(50)

+----------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-----------+--------------------+
|    nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|primaryName|      knownForTitles|
+----------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-----------+--------------------+
|nm11906448|[tt15214950, tt14...|[tvEpisode, tvEpi...|[If Ores Grew On ...|           [0, 0, 0]|     2014|     \N|[Action, Adventur...|   actor|[["Futurian Robot...|       Alex|          tt14187980|
| nm8820071|         [tt6590612]|         [tvEpisode]|[Clowns ohne Gren...|                 [0]|     2015|     \N|            [Comedy]|   actor|[["Clown (Clowns ...|       Alex|           tt3137534|
| nm8